## This notebook shows the overhead in using the inferSchema option when reading in text files in SPARK

### First some intialisation code

In [ ]:

import findspark
findspark.init()
import pyspark
import datetime
sc = pyspark.SparkContext(appName="read-big-file")


from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()




## Time how long it takes in to read an approx 6.5 million record text file using inferSchema

In [ ]:
print(datetime.datetime.now())
# put the the path to your file in the .load statement
df = spark.read.format("com.databricks.spark.csv") \
    .option("header", "false").option("inferSchema", "true") \
    .option("delimiter", '|') \
    .load("file:///d:/tmp/iholding/myfiles/issue50.txt")

print("Nr of records = ", df.count())
print(datetime.datetime.now())
df.printSchema()

## ... and the same without

In [ ]:
print(datetime.datetime.now())
# put the the path to your file in the .load statement
df = spark.read.format("com.databricks.spark.csv") \
    .option("header", "false")\
    .option("delimiter", '|') \
    .load("file:///d:/tmp/iholding/myfiles/issue50.txt")

print("Nr of records = ", df.count())
print(datetime.datetime.now())
df.printSchema()

## If we really wanta  schema - and we usually do - then we can pre-create one
## that fits our data and apply it to the input file
##
## There are two ways of doing this .... do it in code like below

In [ ]:
from pyspark.sql.types import StringType, StructField, \
StructType, DoubleType, LongType,IntegerType, TimestampType

In [ ]:
myschema = StructType([
        StructField("column1", IntegerType(), True),
        StructField("column2", IntegerType(), True),
        StructField("column3", IntegerType(), True),
        StructField("column4", TimestampType(), True),
        StructField("column5", IntegerType(), True),
        StructField("column6", LongType(), True),
        StructField("column7", IntegerType(), True),
        StructField("column8", DoubleType(), True),
        StructField("column9", DoubleType(), True),
        StructField("column10",LongType(), True),
        StructField("column11", DoubleType(), True),
        StructField("column12", IntegerType(), True),
        StructField("column13", IntegerType(), True)
            ])

## now see what timing difference there is compared to the original

In [ ]:
print(datetime.datetime.now())
# put the the path to your file in the .load statement
df = spark.read.format("com.databricks.spark.csv") \
    .option("header", "false").schema(myschema)\
    .option("delimiter", '|') \
    .load("file:///d:/tmp/iholding/myfiles/issue50.txt")

print("Nr of records = ", df.count())
print(datetime.datetime.now())
df.printSchema()

### .... or construct a little dummy text file that matches our real input file, use the inferSchema on it
### and apply it to our real input file

In [ ]:
dummyDf = spark.read.format("com.databricks.spark.csv") \
    .option("header", "false").option("inferSchema","true") \
    .option("delimiter", '|').load("file:///d:/tmp/downloads/tinyfile.txt")

In [ ]:
print(datetime.datetime.now())
# put the the path to your file in the .load statement
df = spark.read.format("csv") \
    .option("header", "false").schema(dummyDf.schema)\
    .option("delimiter", '|') \
    .load("file:///d:/tmp/iholding/myfiles/issue50.txt")
print("Nr of records = ", df.count())
print(datetime.datetime.now())
df.printSchema()